# 1. Pre-processing

#### 1.1. Text Pre-processing

The dataset is already lowercased and lacks punctuation. We will tokenize the text and remove stopwords, as well as apply lemmatization to the tokens.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

data = pd.read_json('./data/data.jsonl', lines=True)
def pre_process_data(dataset):
    #tokenize
    dataset['text'] = dataset['text'].apply(nltk.word_tokenize)

    #remove stop words
    stop_words = set(stopwords.words('english'))
    dataset['text'] = dataset['text'].apply(lambda x: [word for word in x if word not in stop_words])

    #lemmatize
    lemmatizer = WordNetLemmatizer()
    dataset['text'] = dataset['text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

    return dataset

data = pre_process_data(data)
data.head()

In [10]:
test_data = pd.read_json('./data/test.jsonl', lines=True)
train_data = pd.read_json('./data/train.jsonl', lines=True)
validation_data = pd.read_json('./data/validation.jsonl', lines=True)

test_data = pre_process_data(test_data)
train_data = pre_process_data(train_data)
validation_data = pre_process_data(validation_data)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=40000)
X = vectorizer.fit_transform(data['text'].apply(lambda x: ' '.join(x)))
# good idea to use two-grams??
print(X.shape)

(416809, 40000)


In [7]:
print(vectorizer.get_feature_names_out())

['aa' 'aaron' 'ab' ... 'zoom class' 'zoom grey' 'zumba']


In [12]:
X_train = vectorizer.transform(train_data['text'].apply(lambda x: ' '.join(x)))
X_val = vectorizer.transform(validation_data['text'].apply(lambda x: ' '.join(x)))
X_test = vectorizer.transform(test_data['text'].apply(lambda x: ' '.join(x)))

In [13]:
y_train = train_data['label']
y_val = validation_data['label']
y_test = test_data['label']


# 3. Model Training

#### 3.1. Model Selection
Aqui também só estava a querer espetar modelos para começar a ver o que dá que ainda não sei que features vão ser usadas:
tf-idf, word embeddings, ???, features mais feitas à mão?


Isto pelos vistos é uma cena, que não implementei (ainda..)

"The validation set uses a subset of the training data to provide an unbiased evaluation of a model. The validation data set contrasts with training and test sets in that it is an intermediate phase used for choosing the best model and optimizing it. It is in this phase that hyperparameter tuning occurs."

Wikipedia:
The basic process of using a validation data set for model selection (as part of training data set, validation data set, and test data set) is:

Since our goal is to find the network having the best performance on new data, the simplest approach to the comparison of different networks is to evaluate the error function using data which is independent of that used for training. Various networks are trained by minimization of an appropriate error function defined with respect to a training data set. The performance of the networks is then compared by evaluating the error function using an independent validation set, and the network having the smallest error with respect to the validation set is selected. This approach is called the hold out method. Since this procedure can itself lead to some overfitting to the validation set, the performance of the selected network should be confirmed by measuring its performance on a third independent set of data called a test set.

An application of this process is in early stopping, where the candidate models are successive iterations of the same network, and training stops when the error on the validation set grows, choosing the previous model (the one with minimum error).

##### 3.1.1. Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

logreg_classifier = LogisticRegression(max_iter=1000)
logreg_classifier.fit(X_train, y_train)
y_pred = logreg_classifier.predict(X_val)
print(accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred)) 

0.876
              precision    recall  f1-score   support

           0       0.88      0.96      0.92       550
           1       0.84      0.97      0.90       704
           2       0.93      0.63      0.75       178
           3       0.93      0.83      0.87       275
           4       0.89      0.75      0.81       212
           5       0.92      0.57      0.70        81

    accuracy                           0.88      2000
   macro avg       0.90      0.78      0.83      2000
weighted avg       0.88      0.88      0.87      2000

[[526  13   0   5   5   1]
 [  9 682   8   2   2   1]
 [  9  53 113   3   0   0]
 [ 24  19   1 227   4   0]
 [ 19  26   0   7 158   2]
 [  9  16   0   1   9  46]]


##### 3.1.2. Multinomial Naive Bayes

In [17]:

mnb_classifier = MultinomialNB()
mnb_classifier.fit(X_train, y_train)
y_pred = mnb_classifier.predict(X_val)
print(accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))



0.702
              precision    recall  f1-score   support

           0       0.73      0.94      0.82       550
           1       0.64      0.99      0.78       704
           2       1.00      0.08      0.16       178
           3       0.98      0.37      0.53       275
           4       0.92      0.34      0.50       212
           5       1.00      0.02      0.05        81

    accuracy                           0.70      2000
   macro avg       0.88      0.46      0.47      2000
weighted avg       0.79      0.70      0.64      2000

[[517  31   0   1   1   0]
 [  7 697   0   0   0   0]
 [ 22 141  15   0   0   0]
 [ 72 102   0 101   0   0]
 [ 67  72   0   1  72   0]
 [ 28  46   0   0   5   2]]


# 4. Model Evaluation

In [18]:
# evaluate with test set
y_pred = logreg_classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


0.873
              precision    recall  f1-score   support

           0       0.90      0.94      0.92       581
           1       0.84      0.97      0.90       695
           2       0.85      0.54      0.66       159
           3       0.90      0.81      0.85       275
           4       0.91      0.82      0.86       224
           5       0.97      0.42      0.59        66

    accuracy                           0.87      2000
   macro avg       0.89      0.75      0.80      2000
weighted avg       0.88      0.87      0.87      2000

[[549  19   1  10   2   0]
 [  4 675  14   1   0   1]
 [ 10  60  86   3   0   0]
 [ 26  22   0 224   3   0]
 [ 18  12   0  10 184   0]
 [  5  18   0   1  14  28]]


In [19]:
y_pred = mnb_classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


0.7145
              precision    recall  f1-score   support

           0       0.75      0.94      0.83       581
           1       0.65      0.99      0.79       695
           2       1.00      0.07      0.13       159
           3       0.94      0.37      0.53       275
           4       0.91      0.36      0.51       224
           5       1.00      0.03      0.06        66

    accuracy                           0.71      2000
   macro avg       0.87      0.46      0.47      2000
weighted avg       0.79      0.71      0.66      2000

[[544  36   0   0   1   0]
 [  4 691   0   0   0   0]
 [ 28 120  11   0   0   0]
 [ 73 101   0 101   0   0]
 [ 68  70   0   6  80   0]
 [ 13  44   0   0   7   2]]
